# CSP Solver for Wordle game

### Useful links :
- [Medium article](https://medium.com/better-programming/beating-wordle-constraint-programming-ef0b0b6897fe#:~:text=Beating%20Wordle%3A%20Constraint%20Programming,Wordle%20solver%20do%20its%20thing)
- [Sample dataset (GitHub)](https://github.com/dwyl/english-words)


### The CSP solver handles the logical part:
- It takes the current game constraints (green, yellow, grey letters)
- It applies these constraints to filter the dictionary of valid words
- It produces a list of candidate words that satisfy all constraints
- It uses heuristics (letter frequencies, duplicate penalties) to prioritize these candidates
### The LLM handles the strategic part
- It receives the list of CSP-filtered candidates
- It can call functions to evaluate these candidates (like calculating information gain)
- It analyzes which words would be most effective for narrowing down possibilities
- It makes the final decision about which word to guess

In [ ]:
%pip install openai
%pip install constraint
%pip install pandas
%pip install python-dotenv
%pip install python-constraint
%pip install ortools
%pip install numpy


In [73]:
import os
import random
import json
import re
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
import itertools
from typing import Dict, Set, List, Tuple, Optional
from dataclasses import dataclass, field
import openai
from dotenv import load_dotenv
from ortools.sat.python import cp_model
import math

In [74]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_API_KEY is None:
    raise ValueError("The OPENAI_API_KEY environment variable is not set.")

RANDOM_SEED = 42
random.seed(RANDOM_SEED)

In [75]:
@dataclass
class WordleConstraints:
    """Class to store Wordle game constraints"""
    green: Dict[int, str] = field(default_factory=dict)  # Correct letter in correct position
    yellow: Dict[int, str] = field(default_factory=dict)  # Correct letter in wrong position
    grey: Set[str] = field(default_factory=set)  # Letters not in the word
    min_letter_counts: Dict[str, int] = field(default_factory=dict)
    
    def __str__(self):
        return f"Green: {self.green}, Yellow: {self.yellow}, Grey: {self.grey}, Min counts: {self.min_letter_counts}"


In [76]:
class CSPSolver:
    """Constraint Satisfaction Problem solver for Wordle using OR-Tools CP-SAT"""
    def __init__(self, word_length=5):
        self.word_length = word_length
        self.valid_words = []
        
        try:
            words = pd.read_json("words_alpha.txt", orient="records")
            self.words_data = words["words"].to_list()
        except Exception as e:
            print(f"Init Error loading words: {e}")
            self.words_data = []
        
        self._initialize_letter_statistics()
    
    def _initialize_letter_statistics(self):
        """Initialize letter statistics for heuristic function"""
        self.positional_freq = [defaultdict(int) for _ in range(self.word_length)]
        for word in self.words_data:
            for pos in range(self.word_length):
                if pos < len(word):
                    char = word[pos]
                    self.positional_freq[pos][char] += 1

        self.letter_frequency = Counter(itertools.chain.from_iterable(self.words_data))

        total_words = len(self.words_data)
        for char, freq in self.letter_frequency.items():
            self.letter_frequency[char] = freq / total_words

        for pos in range(self.word_length):
            total_pos = sum(self.positional_freq[pos].values())
            if total_pos > 0:
                for char, freq in self.positional_freq[pos].items():
                    self.positional_freq[pos][char] = freq / total_pos
    
    def set_valid_words(self, words):
        """Set list of valid words to filter solutions"""
        self.valid_words = words
        self.words_data = words
        self._initialize_letter_statistics()
    
    def get_solutions(self, constraints, max_solutions=100):
        """Get CSP solutions using OR-Tools CP-SAT"""
        words_as_ints = []
        for word in self.valid_words:
            if len(word) == self.word_length:
                word_as_ints = [ord(c) - ord('a') for c in word]
                words_as_ints.append(tuple(word_as_ints))
        
        if not words_as_ints:
            print("No valid words found")
            return []
        
        model = cp_model.CpModel()
        position_vars = [model.NewIntVar(0, 25, f'pos_{i}') for i in range(self.word_length)]
        
        self._update_model(model, position_vars, constraints)
        
        model.AddAllowedAssignments(position_vars, words_as_ints)
        
        self._update_heuristic(model, position_vars)
        
        solver = cp_model.CpSolver()
        status = solver.Solve(model)
        
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            solution = [solver.Value(var) for var in position_vars]
            solution_word = ''.join([chr(c + ord('a')) for c in solution])
            
            # Get more solutions if needed
            solutions = [solution_word]
            valid_words = set(self.valid_words)
            
            # Simple filtering approach for additional solutions
            for word in self.valid_words:
                if word != solution_word and len(solutions) < max_solutions:
                    if self._is_valid_word_for_constraints(word, constraints):
                        solutions.append(word)
            print("OR-Tools solutions:", solutions)
            return solutions
        else:
            print("No solution found")
            return []
    
    def _is_valid_word_for_constraints(self, word, constraints):
        """Check if a word satisfies all the given constraints"""
        # Check green constraints
        for pos, letter in constraints.green.items():
            if word[pos] != letter:
                return False
        
        # Check yellow constraints
        for pos, letter in constraints.yellow.items():
            if word[pos] == letter or letter not in word:
                return False
        
        # Check grey constraints
        for letter in constraints.grey:
            if letter in word:
                # If the letter is in green or yellow positions, it's allowed
                if letter in constraints.green.values() or letter in constraints.yellow.values():
                    # Check if the count exceeds the min count
                    min_count = constraints.min_letter_counts.get(letter, 0)
                    if word.count(letter) > min_count:
                        return False
                else:
                    return False
        
        # Check minimum letter counts
        for letter, count in constraints.min_letter_counts.items():
            if word.count(letter) < count:
                return False
        
        return True
    
    def _update_model(self, model, position_vars, constraints):
        """Update the model using the constraints"""
        # Apply green constraints
        for pos, letter in constraints.green.items():
            letter_idx = ord(letter) - ord('a')
            model.Add(position_vars[pos] == letter_idx)
        
        # Apply yellow constraints
        for pos, letter in constraints.yellow.items():
            letter_idx = ord(letter) - ord('a')
            model.Add(position_vars[pos] != letter_idx)
            
            # Letter must appear somewhere
            letter_occurs = [model.NewBoolVar(f'yellow_{letter}_{p}') for p in range(self.word_length)]
            for p in range(self.word_length):
                model.Add(position_vars[p] == letter_idx).OnlyEnforceIf(letter_occurs[p])
                model.Add(position_vars[p] != letter_idx).OnlyEnforceIf(letter_occurs[p].Not())
            model.Add(sum(letter_occurs) >= 1)
        
        # Apply grey constraints
        for letter in constraints.grey:
            letter_idx = ord(letter) - ord('a')
            # Skip if the letter is in green or yellow (duplicate letter case)
            if letter in constraints.green.values() or letter in constraints.yellow.values():
                # Limit the letter count to the minimum required
                min_count = constraints.min_letter_counts.get(letter, 0)
                letter_occurs = [model.NewBoolVar(f'grey_{letter}_{p}') for p in range(self.word_length)]
                for p in range(self.word_length):
                    model.Add(position_vars[p] == letter_idx).OnlyEnforceIf(letter_occurs[p])
                    model.Add(position_vars[p] != letter_idx).OnlyEnforceIf(letter_occurs[p].Not())
                model.Add(sum(letter_occurs) == min_count)
            else:
                # Letter must not appear at all
                for p in range(self.word_length):
                    model.Add(position_vars[p] != letter_idx)
    
    def _update_heuristic(self, model, position_vars):
        """Update the heuristic function to maximize for the model"""
        # Define the coefficients for the heuristic
        c_pos_freq = 1000
        c_letter_freq = 100
        c_dup = 100

        # Build the heuristic objective function
        objective = []
        for pos in range(self.word_length):
            char_var = position_vars[pos]
            for char in range(26):
                is_char = model.NewBoolVar(f'pos_{pos}_char_{char}')
                model.Add(char_var == char).OnlyEnforceIf(is_char)
                model.Add(char_var != char).OnlyEnforceIf(is_char.Not())
                
                # Calculate the score based on letter frequencies
                char_letter = chr(char + ord('a'))
                pos_freq = self.positional_freq[pos].get(char_letter, 0)
                letter_freq = self.letter_frequency.get(char_letter, 0)
                score = int(c_pos_freq * pos_freq + c_letter_freq * letter_freq)
                objective.append(is_char * score)
                
        # Calculate the number of duplicates
        num_duplicates = model.NewIntVar(0, 25, 'num_duplicates')
        char_counts = []
        for char in range(26):
            count = model.NewIntVar(0, self.word_length, f'count_{char}')
            letter_occurs = [model.NewBoolVar(f'occurs_{char}_{p}') for p in range(self.word_length)]
            for p in range(self.word_length):
                model.Add(position_vars[p] == char).OnlyEnforceIf(letter_occurs[p])
                model.Add(position_vars[p] != char).OnlyEnforceIf(letter_occurs[p].Not())
            model.Add(count == sum(letter_occurs))
            char_counts.append(count)

        # Add constraints to calculate the number of duplicates
        duplicates = []
        for count in char_counts:
            is_duplicate = model.NewBoolVar(f'is_duplicate_{count.Index()}')
            model.Add(count > 1).OnlyEnforceIf(is_duplicate)
            model.Add(count <= 1).OnlyEnforceIf(is_duplicate.Not())
            duplicates.append(is_duplicate)

        model.Add(num_duplicates == sum(duplicates))

        objective.append(-c_dup * num_duplicates)

        model.Maximize(sum(objective))
    
    def filter_valid_words(self, constraints):
        """Filter all valid words based on the constraints"""
        return [word for word in self.valid_words if self._is_valid_word_for_constraints(word, constraints)]


In [77]:
class LanguageAgent:
    """LLM-based agent for strategic word selection"""
    def __init__(self, model_name="gpt-4o-mini", api_key=None):
        self.model_name = model_name
        self.client = openai.OpenAI(api_key=api_key)
        self.past_guesses = []
        self.last_explanation = None
    
    def generate_feedback(self, guess, target_word):
        """Generate feedback for a guess in Wordle format"""
        if len(guess) != len(target_word):
            print(f"Error: Guess '{guess}' and target '{target_word}' must be of same length")
            return None
        
        # Count letters in target word for proper yellow handling
        target_letter_counts = {}
        for letter in target_word:
            target_letter_counts[letter] = target_letter_counts.get(letter, 0) + 1
        
        # First pass: Mark green matches and decrement counts
        feedback = [''] * len(guess)
        for i, (g, t) in enumerate(zip(guess, target_word)):
            if g == t:
                feedback[i] = 'green'
                target_letter_counts[g] -= 1
        
        # Second pass: Mark yellow and grey
        for i, (g, f) in enumerate(zip(guess, feedback)):
            if f == '':  # Not marked green yet
                if g in target_letter_counts and target_letter_counts[g] > 0:
                    feedback[i] = 'yellow'
                    target_letter_counts[g] -= 1
                else:
                    feedback[i] = 'grey'
                    
        return feedback
    
    def _calculate_entropy(self,word_list):
        total = len(word_list)
        if total == 0:
            return 0
        return math.log2(total)

    def _calculate_information_gain(self, word, word_candidates):
        if not word or not word_candidates:
            return 0.0
        
        initial_entropy = self._calculate_entropy(word_candidates)
        
        feedback_groups = defaultdict(list)
        
        for potential_target in word_candidates:
            feedback = self.generate_feedback(word, potential_target)
            feedback_key = tuple(feedback)  # Convert list to hashable tuple
            feedback_groups[feedback_key].append(potential_target)
        
        # Calculate expected entropy after guess
        expected_entropy = 0.0
        total = len(word_candidates)
        
        for feedback, group in feedback_groups.items():
            probability = len(group) / total
            group_entropy = self._calculate_entropy(group)
            expected_entropy += probability * group_entropy
        
        # Information gain = reduction in entropy
        information_gain = initial_entropy - expected_entropy
        return round(information_gain, 3)
    
    def _extract_word(self, content, candidates, word_length):
        """Extract a word from the LLM's final response"""
        # First try to find an exact word match
        words = re.findall(r'\b[a-zA-Z]{%d}\b' % word_length, content.lower())
        for word in words:
            if word in candidates:
                return word
        
        # If no direct match, look for a word wrapped in quotes or formatting
        quoted_words = re.findall(r'["\'`]([a-zA-Z]{%d})["\'`]' % word_length, content.lower())
        for word in quoted_words:
            if word in candidates:
                return word
        
        # Last resort - find any word that's in candidates
        for candidate in candidates:
            if candidate.lower() in content.lower():
                return candidate
        
        return None
    
    def suggest_word(self, word_candidates, constraints, word_length, past_guesses=[]):
        """Suggest the best word using function calling"""

        functions = [
            {
                "name": "explain_choice",
                "description": "Explain the reasoning behind choosing a particular word",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "word": {
                            "type": "string",
                            "description": "The chosen word"
                        },
                        "explanation": {
                            "type": "string",
                            "description": "Detailed explanation of why this word is the best choice"
                        }
                    },
                    "required": ["word", "explanation"]
                }
            },
            {
                "name": "filter_by_constraints",
                "description": "Filter words based on Wordle constraints",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "constraints": {
                            "type": "object",
                            "description": "Current game constraints"
                        }
                    },
                    "required": ["constraints"]
                }
            },
            {
                "name": "evaluate_information_gain",
                "description": "Calculate information gain for a potential guess",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "word": {
                            "type": "string",
                            "description": "Word to evaluate"
                        },
                        "remaining_candidates": {
                            "type": "integer",
                            "description": "Number of remaining candidates"
                        }
                    },
                    "required": ["word"]
                }
            }
        ]
        
        prompt = f"""
        You are a Wordle-solving assistant that can call functions to help solve the puzzle.

        Current game state:
        - Word length: {word_length}
        - Previous guesses: {', '.join(past_guesses)}
        - Number of candidates: {len(word_candidates)}

        Analyze the current state, call functions to evaluate words, and suggest the best next guess.
        Before returning your final answer, call the explain_choice function to explain your reasoning.
        Only suggest words that are in the candidates list.
        """
        
        messages = [{"role": "system", "content": prompt}]
        
        print("LLM will choose from :", word_candidates)
        for _ in range(5):  # Limit iterations
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                functions=functions,
                function_call="auto"
            )
            
            response_message = response.choices[0].message
            messages.append(response_message)
            
            # Handle function calls
            if hasattr(response_message, "function_call") and response_message.function_call:
                function_name = response_message.function_call.name
                function_args = json.loads(response_message.function_call.arguments)
                
                if function_name == "filter_by_constraints":
                    result = {"candidates": word_candidates, "total": len(word_candidates)}
                    
                elif function_name == "evaluate_information_gain":
                    word = function_args.get("word")
                    score = self._calculate_information_gain(word, word_candidates)
                    result = {"word": word, "info_gain": score}

                elif function_name == "explain_choice":
                    word = function_args.get("word")
                    explanation = function_args.get("explanation")
                    self.last_explanation = explanation
                    result = {"word": word, "explanation": explanation}
                    print(f"Explanation for choosing '{word}':\n{explanation}")

                messages.append({
                    "role": "function",
                    "name": function_name, 
                    "content": json.dumps(result)
                })
            else:
                suggestion = self._extract_word(response_message.content, word_candidates, word_length)
                if suggestion and suggestion in word_candidates:
                    print("The suggestion is :", suggestion)
                    return suggestion
                else:
                    return random.choice(word_candidates)
        
        return random.choice(word_candidates)


In [78]:
class HybridSolver:
    """Combined CSP + LLM solver for Wordle"""
    def __init__(self, word_length=5, api_key=OPENAI_API_KEY, model_name="gpt-4o-mini"):
        self.word_length = word_length
        self.csp_solver = CSPSolver(word_length)
        self.language_agent = LanguageAgent(model_name, api_key)
        self.constraints = WordleConstraints()
        self.past_guesses = []
        
    def load_words(self, file_path="words_alpha.txt"):
        """Load and set valid words of specified length"""
        words = []
        try:
            with open(file_path, 'r') as f:
                for line in f:
                    word = line.strip().lower()
                    if len(word) == self.word_length:
                        words.append(word)
        except Exception as e:
            print(f"Error loading words: {e}")
            return []
        
        self.csp_solver.set_valid_words(words)
        return words
    
    def generate_word_to_guess(self, word_list):
        """Select a random word to guess"""
        valid_words = [word for word in word_list if len(word) == self.word_length]
        if valid_words:
            word = random.choice(valid_words)
            return word
        else:
            print(f"No valid {self.word_length}-letter words found.")
            return None
        
    def generate_feedback(self, guess, target_word):
        """Generate feedback for a guess in Wordle format"""
        if len(guess) != len(target_word):
            print(f"Error: Guess '{guess}' and target '{target_word}' must be of same length")
            return None
        
        # Count letters in target word for proper yellow handling
        target_letter_counts = {}
        for letter in target_word:
            target_letter_counts[letter] = target_letter_counts.get(letter, 0) + 1
        
        # First pass: Mark green matches and decrement counts
        feedback = [''] * len(guess)
        for i, (g, t) in enumerate(zip(guess, target_word)):
            if g == t:
                feedback[i] = 'green'
                target_letter_counts[g] -= 1
        
        # Second pass: Mark yellow and grey
        for i, (g, f) in enumerate(zip(guess, feedback)):
            if f == '':  # Not marked green yet
                if g in target_letter_counts and target_letter_counts[g] > 0:
                    feedback[i] = 'yellow'
                    target_letter_counts[g] -= 1
                else:
                    feedback[i] = 'grey'
                    
        return feedback
        
    def update_constraints(self, guess, feedback):
        """Update constraints based on feedback"""
        # Track letter counts for handling duplicate letters correctly
        letter_counts = {}
        
        # First pass: update green constraints and count occurrences
        for i, (letter, result) in enumerate(zip(guess, feedback)):
            if result == 'green':
                self.constraints.green[i] = letter
                letter_counts[letter] = letter_counts.get(letter, 0) + 1
        
        # Second pass: update yellow constraints
        for i, (letter, result) in enumerate(zip(guess, feedback)):
            if result == 'yellow':
                self.constraints.yellow[i] = letter
                letter_counts[letter] = letter_counts.get(letter, 0) + 1
        
        # Update minimum letter counts based on green + yellow occurrences
        for letter, count in letter_counts.items():
            current_min = self.constraints.min_letter_counts.get(letter, 0)
            self.constraints.min_letter_counts[letter] = max(current_min, count)
        
        # Third pass: update grey constraints
        for i, (letter, result) in enumerate(zip(guess, feedback)):
            if result == 'grey':
                # Only add to grey set if letter doesn't appear elsewhere 
                # or if we've seen the max number of this letter already
                if letter not in letter_counts:
                    self.constraints.grey.add(letter)
                    
    def _generate_fallback_guess(self):
        """Generate a guess when CSP finds no solutions"""
        # Start with common starter words not tried yet
        starters = ["crane", "adieu", "slice", "tried", "plane", "roast"]
        for word in starters:
            if word not in self.past_guesses:
                # Check if it respects green letters at least
                valid = True
                for pos, letter in self.constraints.green.items():
                    if word[pos] != letter:
                        valid = False
                        break
                if valid:
                    return word
        
        # If no starter works, generate a word satisfying green constraints
        template = ['_'] * self.word_length
        for pos, letter in self.constraints.green.items():
            template[pos] = letter
        
        # Fill remaining positions with common letters
        common_letters = "etaoinshrdlucmfwypvbgkjqxz"
        for i in range(self.word_length):
            if template[i] == '_':
                # Try letters that aren't in grey set
                for letter in common_letters:
                    if letter not in self.constraints.grey:
                        template[i] = letter
                        break
        
        return ''.join(template)
    
    def get_next_guess(self):
        """Generate the next guess using hybrid approach"""
        # For first 2-3 guesses, use predetermined good starting words to save time
        if (len(self.past_guesses) == 0) and (self.word_length == 5):
            guess = "crane" # I saw in a video that this is a good starting word when the word length is 5
            self.past_guesses.append(guess)
            return guess
        elif len(self.past_guesses) == 1 and (self.word_length == 5):
            # Second guess depends on first guess results
            green_count = sum(1 for pos, letter in self.constraints.green.items())
            yellow_count = sum(1 for pos, letter in self.constraints.yellow.items())
            
            if green_count + yellow_count <= 1:
                # First guess didn't reveal much, use complementary word
                guess = "solid"  # Covers most common letters not in "crane"
                self.past_guesses.append(guess)
                return guess
        
        # Get valid word candidates from CSP
        candidates = self.csp_solver.get_solutions(self.constraints, max_solutions=1000)
        print(f"CSP found {len(candidates)} candidate words")

        if candidates:
            guess = self.language_agent.suggest_word(
                candidates, 
                self.constraints, 
                self.word_length,
                self.past_guesses
            )
            self.past_guesses.append(guess)
            return guess
        else:
            print("No candidates found, trying to relax constraints...")
            return self._generate_fallback_guess()
    
    def solve_wordle(self, target_word=None, word_list=None, max_attempts=6):
        """Main method to solve Wordle"""
        if word_list is None:
            word_list = self.load_words()
        
        if not target_word:
            target_word = self.generate_word_to_guess(word_list)
        
        if not target_word:
            return None
        
        print(f"Starting to solve for: {target_word}")
        
        self.constraints = WordleConstraints()
        self.past_guesses = []
        
        for attempt in range(max_attempts):
            print(f"\nAttempt {attempt+1}:")
            
            guess = self.get_next_guess()
            if not guess:
                print("Failed to generate a valid guess")
                break
                
            print(f"Guess: {guess}")
            
            feedback = self.generate_feedback(guess, target_word)
            print(f"Feedback: {feedback}")
            
            if all(f == 'green' for f in feedback):
                print(f"\nSolved in {attempt+1} attempts! The word is: {guess}")
                return attempt+1
            
            self.update_constraints(guess, feedback)
            print(f"Updated constraints: {self.constraints}")
        
        print(f"\nFailed to solve in {max_attempts} attempts. The word was: {target_word}")
        return None

In [79]:
solver = HybridSolver(word_length=5, api_key=OPENAI_API_KEY)
words = solver.load_words()

# Solve a single Wordle
solver.solve_wordle(word_list=words)

# Or run a benchmark
# benchmark(solver, words_to_test=5) 

Init Error loading words: Expected object or value
Starting to solve for: peppy

Attempt 1:
Guess: crane
Feedback: ['grey', 'grey', 'grey', 'grey', 'yellow']
Updated constraints: Green: {}, Yellow: {4: 'e'}, Grey: {'n', 'a', 'r', 'c'}, Min counts: {'e': 1}

Attempt 2:
Guess: solid
Feedback: ['grey', 'grey', 'grey', 'grey', 'grey']
Updated constraints: Green: {}, Yellow: {4: 'e'}, Grey: {'i', 'd', 'o', 'r', 'n', 'a', 'l', 's', 'c'}, Min counts: {'e': 1}

Attempt 3:
OR-Tools solutions: ['puget', 'beefy', 'beeth', 'beety', 'begem', 'beget', 'begum', 'begut', 'behew', 'betty', 'bevvy', 'bewet', 'ebbet', 'effet', 'egypt', 'eyght', 'emmet', 'emmew', 'empty', 'expwy', 'fezzy', 'fumet', 'gehey', 'gemmy', 'getup', 'heezy', 'hefty', 'heygh', 'hempy', 'hetty', 'heugh', 'hewgh', 'humet', 'yeuky', 'yezzy', 'yquem', 'jehup', 'jemez', 'jemmy', 'jetty', 'kebby', 'kempy', 'kempt', 'ketty', 'meeth', 'meggy', 'peepy', 'peggy', 'peppy', 'petty', 'petum', 'pumex', 'queet', 'quegh', 'tebet', 'teeth', 'teety

5